# Алгоритмы анализа данных


> [Geek University Data Analytics](https://geekbrains.ru/)

##  [ Урок 2. Масштабирование признаков. L1- и L2-регуляризация. Стохастический градиентный спуск](https://geekbrains.ru/lessons/108932/)

## Задание:


<b>Домашнее задание</b>

1.У вас, с прошлого урока, имеются реализации расчёта среднеквадратичной ошибки и её градиента для линейнй регрессии с коэффициентами при факторах(w) и свободным коэффициентам.

```python
class linear_regression:
    def _mserror(self, X, y_real):
        #рассчёт среднеквадратичной ошибки
        y = X.dot(self.w.T)+self.w0
        return np.sum((y - y_real)**2) / y_real.shape[0]
    def _mserror_grad(self, X, y_real):
        #рассчёт градиента ошибки.
        #2*delta.T.dot(X)/y_real.shape[0] - градиент по коэффициентам при факторах
        #np.sum(2*delta)/y_real.shape[0] - производная(градиент) при нулевом коэффициенте
        delta=(X.dot(self.w.T)+self.w0-y_real)
        return 2*delta.T.dot(X)/y_real.shape[0], np.sum(2*delta)/y_real.shape[0]
```

В этом задании вы должны модифицировать реализацию рассчёта среднеквадратичной ошибки и рассчёта её производной, так, чтобы с к среднеквадратичной ошибке добавлялась l2 регулярязационная поправка: $ +c*\sum \limits _{j}  w_{j}^2 $ а к градиенту- ссответствующее выражение для градиента регулярязационной поправки. 

##### Изначальный класс:

In [42]:
class linear_regression:
    def __init__(self, eta = 0.9, max_iter = 1e4, min_weight_dist = 1e-8):
        self.eta = eta
        self.max_iter = max_iter
        self.min_weight_dist = min_weight_dist
    def _mserror(self, X, y_real):
        #рассчёт среднеквадратичной ошибки
        y = X.dot(self.w.T)+self.w0
        return np.sum((y - y_real)**2) / y_real.shape[0]
    def _mserror_grad(self, X, y_real):
        #рассчёт градиента ошибки.
        #2*delta.T.dot(X)/y_real.shape[0] - градиент по коэффициентам при факторах
        #np.sum(2*delta)/y_real.shape[0] - производная(градиент) при нулевом коэффициенте
        delta=(X.dot(self.w.T)+self.w0-y_real)
        return 2*delta.T.dot(X)/y_real.shape[0], np.sum(2*delta)/y_real.shape[0]
    def _optimize(self, X, Y):
        #оптимизация коэффициентов
        iter_num = 0
        weight_dist = np.inf
        self.w = np.zeros((1, X.shape[1]))
        self.w0=0
        while weight_dist > self.min_weight_dist and iter_num < self.max_iter:
            gr_w, gr_w0=self._mserror_grad(X, Y)
            if iter_num==0:
                #Чтобы eta адаптировалась к порядку градиента, делим на l2 норму градиента в нуле
                eta=self.eta/np.sqrt(np.linalg.norm(gr_w)**2+(gr_w0)**2)
            new_w = self.w - eta * gr_w
            new_w0= self.w0 - eta * gr_w0
            weight_dist = np.sqrt(np.linalg.norm(new_w - self.w)**2+(new_w0 - self.w0)**2)
            iter_num += 1
            self.w = new_w
            self.w0 = new_w0
    def fit(self, X, Y):
        if Y.ndim==1:
            Y=Y[:, np.newaxis]
        self._optimize(X, Y)
    def predict(self, X):
        return (X.dot(self.w.T)+self.w0).flatten()
    def test(self, X, Y):
        if Y.ndim==1:
            Y=Y[:, np.newaxis]
        return self._mserror(X, Y)

Переназначаем класс. Для этого нужно ввести лямбду. Лямбда взята из [примера](https://github.com/annulet/DS_Algorithms/blob/master/hw_2.ipynb)

In [48]:
class new_lin_reg(linear_regression):
    def _mserror(self, X, y_real, lambda_=1e-8):
        y = X.dot(self.w.T)+self.w0
        res = np.sum((y - y_real)**2) / y_real.shape[0] + lambda_* self.w**2
        return res   
    
    def _mserror_grad(self, X, y_real):
        '''
        Рассчёт градиента ошибки. Градиент ошибки - по сути производная от функции, 
        описывающей ошибку.
        '''
        delta =(X.dot(self.w.T)+self.w0-y_real)
        O1 = 2*delta.T.dot(X)/y_real.shape[0] 
        O0 = np.sum(2*delta)/y_real.shape[0]
        ## reg_grad = 
        
        return O1, O0

Я понимаю, что здесь должна быть формула, показывающая скорость изменения регулизационной ошибки. Но как её описать - не смог придумать.

2.На основе этих функций создайте свою регуляризированную полиномиальную регрессию и опробуйте на одном из примеров построения полиномиальной модели из этого урока. <br>


<i><b>Пояснение:</b> Для этого Вам достаточно создать класс, который наследуется от класса polynomial_regression из данного урока, и переопределить в нём методы mserror, mserror_grad(под переопределением подразумевается создание на новом классе методов с таким же названием). </i>


***

In [14]:
shift=np.random.uniform(0, 100)
shift

3.0772939735436267

In [32]:
# Возьмем 2 признака и 1000 объектов
n_features = 4
n_objects = 1000

# сгенерируем вектор истинных весов
w_true = np.random.normal(size=(1, n_features ))

# сгенерируем матрицу X, вычислим Y с добавлением случайного шума
X = np.random.uniform(-7, 7, (n_objects, n_features))
Y = X.dot(w_true.T) + np.random.normal(0, 0.5, size=(n_objects, 1))

# возьмем нулевые начальные веса
w = np.zeros((1, n_features))

In [49]:
shift=np.random.uniform(0, 100)
Y_shift=Y+shift
lr=new_lin_reg()
lr.fit(X, Y_shift)
lr.test(X, Y_shift)

array([[0.25529714, 0.25529716, 0.25529714, 0.25529714]])

In [50]:
shift, lr.w0

(41.16315379691662, 41.17004064787974)

In [51]:
lr.w, w_true

(array([[-0.34035137,  1.32720351, -0.07548054,  0.52864583]]),
 array([[-0.34842471,  1.32168038, -0.08148227,  0.52951858]]))

# Заметки

[Статья на Хабре](https://habr.com/ru/company/nix/blog/425907/#масштабирование)

[Статья про L1, L2 регулизацию](https://craftappmobile.com/l1-%D0%B8-l2-%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F-%D0%B4%D0%BB%D1%8F-%D0%BB%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B9-%D1%80/)